# Practical Exercise 6.02: Comparing fund performance with benchmark performance.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import empyrical as ep  # Importing empyrical for financial metrics

# Sample data: Simulated daily returns of a fund and benchmark
np.random.seed(42)
dates = pd.date_range(start='2025-01-01', periods=252, freq='B')
fund_returns = np.random.normal(0.0005, 0.01, len(dates))  # Simulating daily fund returns
benchmark_returns = np.random.normal(0.0004, 0.009, len(dates))  # Simulating benchmark returns

# Convert to pandas DataFrame
data = pd.DataFrame({'Date': dates, 'Fund_Returns': fund_returns, 'Benchmark_Returns': benchmark_returns})
data.set_index('Date', inplace=True)

# Risk-free rate (daily rate)
risk_free_rate = 0.02 / 252  # Convert annual rate to daily

# Function to calculate key metrics using empyrical
def calculate_metrics(df, risk_free_rate):
    fund = df['Fund_Returns']
    benchmark = df['Benchmark_Returns']
    excess_returns = fund - benchmark  # Required for tracking error and information ratio

    annualized_return = ep.annual_return(fund)
    benchmark_annualized_return = ep.annual_return(benchmark)
    annualized_volatility = ep.annual_volatility(fund)
    sharpe_ratio = ep.sharpe_ratio(fund, risk_free=risk_free_rate)
    sortino_ratio = ep.sortino_ratio(fund, required_return=risk_free_rate)
    max_drawdown = ep.max_drawdown(fund)

    # FIX: Manually calculate Tracking Error
    tracking_error = excess_returns.std() * np.sqrt(252)

    # FIX: Manually calculate Information Ratio
    excess_annualized_return = annualized_return - benchmark_annualized_return
    information_ratio = excess_annualized_return / tracking_error if tracking_error != 0 else np.nan

    beta = ep.beta(fund, benchmark)
    alpha = ep.alpha(fund, benchmark, risk_free=risk_free_rate)
    calmar_ratio = ep.calmar_ratio(fund)

    return {
        'Annualized Return': annualized_return,
        'Benchmark Annualized Return': benchmark_annualized_return,
        'Annualized Volatility': annualized_volatility,
        'Sharpe Ratio': sharpe_ratio,
        'Sortino Ratio': sortino_ratio,
        'Maximum Drawdown': max_drawdown,
        'Tracking Error': tracking_error,  # FIXED: Now calculated manually
        'Information Ratio': information_ratio,  # FIXED: Now calculated manually
        'Beta': beta,
        'Alpha': alpha,
        'Calmar Ratio': calmar_ratio
    }

# Calculate metrics
metrics = calculate_metrics(data, risk_free_rate)

# Display results
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

# Plot Fund vs. Benchmark Returns Over Time
plt.figure(figsize=(10, 5))
plt.plot(data.index, data['Fund_Returns'].cumsum(), label='Fund', color='blue')
plt.plot(data.index, data['Benchmark_Returns'].cumsum(), label='Benchmark', color='orange')
plt.xlabel("Date")
plt.ylabel("Cumulative Returns")
plt.title("Comparison of Fund and Benchmark Cumulative Returns")
plt.legend()
plt.show()
